In [17]:
import json
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel

In [18]:
with open('/home/workspace/xxx/RLMRec_trans/data/steam/usr_prf.pkl', 'rb') as f:
    usr_prf = pickle.load(f)
print(len(usr_prf))

23310


In [19]:
print(usr_prf[0])

{'profile': 'This user is likely to enjoy fast-paced and addictive arcade games, puzzle games with a touch of strategy and comedy, and unique first-person action adventure games with parkour mechanics and immersive atmospheres.', 'reasoning': "Based on the games the user has played, they have shown interest in casual and challenging indie games like Timberman, as well as puzzle games like Tiny Bridge: Ratventure. Additionally, the user has interacted with Mirror's Edge, which suggests they enjoy first-person action adventure games with unique mechanics and immersive worlds."}


In [20]:
print(type(usr_prf))

<class 'dict'>


In [21]:
with open('/home/workspace/xxx/RLMRec_trans/data/steam/itm_prf.pkl', 'rb') as f:
    itm_prf = pickle.load(f)
print(len(itm_prf))


5237


In [22]:
print(itm_prf[0])

{'profile': "Users who enjoy addictive puzzle-matching games with RPG elements and a challenging progression system may enjoy Gunspell - Steam Edition. However, the game's poor grammar and lack of sound options may deter some users.", 'reasoning': "The feedback from users suggests that Gunspell offers an addictive and fun gameplay experience. The mention of a complex system of items, magic, and fighting tactics indicates that players who enjoy strategic RPG elements may find the game appealing. The feedback also mentions that the game becomes more challenging as the hero levels up, which suggests a sense of progression and difficulty. However, there are complaints about the game's poor grammar and lack of sound options, which may be off-putting to some users. Overall, Gunspell seems to appeal to players who enjoy puzzle-matching games with RPG elements and are willing to overlook its shortcomings."}


In [23]:
model = AutoModel.from_pretrained('/home/workspace/xxx/LLMs/llama2-embedding', trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('/home/workspace/xxx/LLMs/llama2-embedding')

In [24]:
_ = model.cuda()

In [25]:
usr_repre_np = np.zeros((len(usr_prf), 1536))

In [26]:
for i in range(len(usr_prf)):
    padded = tokenizer([usr_prf[i]['profile']], return_tensors="pt", padding=True)
    for k_ in padded: # input_ids, attention_mask
        padded[k_] = padded[k_].cuda()
    cur_repre = model.encode(padded).cpu().detach().numpy()[0]
    usr_repre_np[i] = cur_repre


In [27]:
print(usr_repre_np.shape)

(23310, 1536)


In [28]:
pickle.dump(usr_repre_np, open('/home/workspace/xxx/RLMRec_trans/data/steam/usr_repre_np.pkl', 'wb'))

In [29]:
itm_repre_np = np.zeros((len(itm_prf), 1536))

In [30]:
for i in range(len(itm_prf)):
    padded = tokenizer([itm_prf[i]['profile']], return_tensors="pt", padding=True)
    for k_ in padded: # input_ids, attention_mask
        padded[k_] = padded[k_].cuda()
    cur_repre = model.encode(padded).cpu().detach().numpy()[0]
    itm_repre_np[i] = cur_repre


In [31]:
pickle.dump(itm_repre_np, open('/home/workspace/xxx/RLMRec_trans/data/steam/itm_repre_np.pkl', 'wb'))